# **💁🏻🗨️💁🏻‍♂️대화 요약 Baseline code**
> **Dialogue Summarization** 경진대회에 오신 여러분 환영합니다! 🎉    
> 본 대회에서는 최소 2명에서 최대 7명이 등장하여 나누는 대화를 요약하는 BART 기반 모델의 baseline code를 제공합니다.     
> 주어진 데이터를 활용하여 일상 대화에 대한 요약을 효과적으로 생성하는 모델을 만들어봅시다!

## ⚙️ 데이터 및 환경설정

### 1) 필요한 라이브러리 설치

- 필요한 라이브러리를 설치한 후 불러옵니다.

In [1]:
import pandas as pd
import os
import re
import json
import yaml
from glob import glob
from tqdm import tqdm
from pprint import pprint
import torch
import pytorch_lightning as pl
from rouge import Rouge # 모델의 성능을 평가하기 위한 라이브러리입니다.

from torch.utils.data import Dataset , DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaConfig
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Config
from transformers import T5Tokenizer
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

import wandb # 모델 학습 과정을 손쉽게 Tracking하고, 시각화할 수 있는 라이브러리입니다.

### 2) Config file 만들기 (선택)
- 모델 생성에 필요한 다양한 매개변수 정보를 저장할 수 있습니다.  
  따라서, 코드 상에서 모델의 매개변수를 설정할 수도 있지만 독립적인 매개변수 정보 파일을 생성하여 관리할 수 있습니다.

In [2]:
# config 설정에 tokenizer 모듈이 사용되므로 미리 tokenizer를 정의해줍니다.
tokenizer = AutoTokenizer.from_pretrained("beomi/llama-2-ko-7b")

In [3]:
# config_data = {
#     "general": {
#         "data_path": "../data/", # 모델 생성에 필요한 데이터 경로를 사용자 환경에 맞게 지정합니다.
#         "model_name": "eenzeenee/t5-small-korean-summarization", # 불러올 모델의 이름을 사용자 환경에 맞게 지정할 수 있습니다.
#         "output_dir": "../output/t5_type08/" # 모델의 최종 출력 값을 저장할 경로를 설정합니다.
#     },
#     "tokenizer": {
#         "encoder_max_len": 512,
#         "decoder_max_len": 100,
#         "bos_token": f"{tokenizer.bos_token}",
#         "eos_token": f"{tokenizer.eos_token}",
#         # 특정 단어들이 분해되어 tokenization이 수행되지 않도록 special_tokens을 지정해줍니다.
#         "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
#     },
#     "training": {
#         "overwrite_output_dir": True,
#         "num_train_epochs": 20,
#         "learning_rate": 1e-5,
#         "per_device_train_batch_size": 50,
#         "per_device_eval_batch_size": 32,
#         "warmup_ratio": 0.2,
#         "weight_decay": 0.02,
#         "lr_scheduler_type": 'cosine',
#         "optim": 'adamw_torch',
#         "gradient_accumulation_steps": 1,
#         "evaluation_strategy": 'epoch',
#         "save_strategy": 'epoch',
#         "save_total_limit": 5,
#         "fp16": True,
#         "load_best_model_at_end": True,
#         "seed": 42,
#         "logging_dir": "./logs",
#         "logging_strategy": "epoch",
#         "predict_with_generate": True,
#         "generation_max_length": 100,
#         "do_train": True,
#         "do_eval": True,
#         "early_stopping_patience": 3,
#         "early_stopping_threshold": 0.001,
#         "report_to": "wandb" # (선택) wandb를 사용할 때 설정합니다.
#     },
#     # (선택) wandb 홈페이지에 가입하여 얻은 정보를 기반으로 작성합니다.
#     "wandb": {
#         "entity": "wandb_repo",
#         "project": "project_name",
#         "name": "run_name"
#     },
#     "inference": {
#         "ckt_path": "model ckt path", # 사전 학습이 진행된 모델의 checkpoint를 저장할 경로를 설정합니다.
#         "result_path": "./prediction/",
#         "no_repeat_ngram_size": 2,
#         "early_stopping": True,
#         "generate_max_length": 100,
#         "num_beams": 4,
#         "batch_size" : 32,
#         # 정확한 모델 평가를 위해 제거할 불필요한 생성 토큰들을 정의합니다.
#         "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
#     }
# }

In [4]:
config_data = {
    "general": {
        "data_path": "../data/",  # 모델 학습에 사용할 데이터 경로
        "model_name": "beomi/llama-2-ko-7b",  # 사용할 LLaMA 모델 이름
        "output_dir": "../output/llama_type01/"  # 모델의 출력 값을 저장할 경로
    },
    "tokenizer": {
        "max_length": 300,  # 입력 시퀀스의 최대 길이
        "bos_token": f"{tokenizer.bos_token}",  # LLaMA의 시작 토큰 (필요에 따라 설정)
        "eos_token": f"{tokenizer.eos_token}",  # LLaMA의 종료 토큰 (필요에 따라 설정)
        # 특정 단어들이 분해되지 않도록 special_tokens 설정
        "special_tokens": ['#Person1#', '#Person2#', '#Person3#', '#PhoneNumber#', '#Address#', '#PassportNumber#']
    },
    "training": {
        "overwrite_output_dir": True,
        "num_train_epochs": 20,
        "learning_rate": 1e-5,
        "per_device_train_batch_size": 2,  # 배치 크기는 GPU 메모리 제한에 맞게 조정
        "gradient_accumulation_steps": 4,  # 실질적인 배치 크기를 8로 증가시킴
        "per_device_eval_batch_size": 2,
        "warmup_ratio": 0.1,
        "weight_decay": 0.01,
        "lr_scheduler_type": 'cosine',
        "optim": 'adamw_torch',
        "gradient_accumulation_steps": 1,
        "evaluation_strategy": 'epoch',
        "save_strategy": 'epoch',
        "save_total_limit": 5,
        "fp16": True,  # 혼합 정밀도 사용
        "load_best_model_at_end": True,
        "seed": 42,
        "logging_dir": "./logs",
        "logging_strategy": "epoch",
        "do_train": True,
        "do_eval": True,
        "early_stopping_patience": 3,
        "early_stopping_threshold": 0.001,
        "report_to": "wandb"  # (선택) wandb 사용 시 설정
    },
    # (선택) wandb 설정
    "wandb": {
        "entity": "wandb_repo",
        "project": "project_name",
        "name": "run_name"
    },
    "inference": {
        "ckt_path": "model ckt path",  # 학습된 모델 체크포인트 경로
        "result_path": "./prediction/",
        "no_repeat_ngram_size": 2,  # 생성된 텍스트에서 반복 방지 설정
        "early_stopping": True,
        "max_new_tokens": 100,  # 새로 생성할 최대 토큰 수
        "num_beams": 4,  # 빔 서치 설정
        "batch_size": 4,
        # 불필요한 생성 토큰을 제거하기 위한 설정
        "remove_tokens": ['<usr>', f"{tokenizer.bos_token}", f"{tokenizer.eos_token}", f"{tokenizer.pad_token}"]
    }
}

- 참고✅    
: wandb 라이브러리를 사용하기 위해선 entity, project, name를 지정해주어야 합니다. wandb 홈페이지에 가입한 후 얻은 정보를 입력하여 작동할 수 있습니다.

In [5]:
# 모델의 구성 정보를 YAML 파일로 저장합니다.
config_path = "./config.yaml"
with open(config_path, "w") as file:
    yaml.dump(config_data, file, allow_unicode=True)

### 3) Configuration 불러오기

In [6]:
# 저장된 config 파일을 불러옵니다.
config_path = "./config.yaml"

with open(config_path, "r") as file:
    loaded_config = yaml.safe_load(file)

# 불러온 config 파일의 전체 내용을 확인합니다.
pprint(loaded_config)

{'general': {'data_path': '../data/',
             'model_name': 'beomi/llama-2-ko-7b',
             'output_dir': '../output/llama_type01/'},
 'inference': {'batch_size': 4,
               'ckt_path': 'model ckt path',
               'early_stopping': True,
               'max_new_tokens': 100,
               'no_repeat_ngram_size': 2,
               'num_beams': 4,
               'remove_tokens': ['<usr>', '<s>', '</s>', '</s>'],
               'result_path': './prediction/'},
 'tokenizer': {'bos_token': '<s>',
               'eos_token': '</s>',
               'max_length': 300,
               'special_tokens': ['#Person1#',
                                  '#Person2#',
                                  '#Person3#',
                                  '#PhoneNumber#',
                                  '#Address#',
                                  '#PassportNumber#']},
 'training': {'do_eval': True,
              'do_train': True,
              'early_stopping_patience': 3,
         

In [7]:
# 실험에 쓰일 데이터의 경로, 사용될 모델, 모델의 최종 출력 결과를 저장할 경로에 대해 확인합니다.
loaded_config['general']

{'data_path': '../data/',
 'model_name': 'beomi/llama-2-ko-7b',
 'output_dir': '../output/llama_type01/'}

In [8]:
# 이곳에 사용자가 저장한 데이터 dir 설정하기
# loaded_config['general']['data_path'] = "data_path"

In [9]:
# 데이터 전처리를 하기 위해 tokenization 과정에서 필요한 정보들을 확인합니다.
loaded_config['tokenizer']

{'bos_token': '<s>',
 'eos_token': '</s>',
 'max_length': 300,
 'special_tokens': ['#Person1#',
  '#Person2#',
  '#Person3#',
  '#PhoneNumber#',
  '#Address#',
  '#PassportNumber#']}

In [10]:
# 모델이 훈련 시 적용될 매개변수를 확인합니다.
loaded_config['training']

{'do_eval': True,
 'do_train': True,
 'early_stopping_patience': 3,
 'early_stopping_threshold': 0.001,
 'evaluation_strategy': 'epoch',
 'fp16': True,
 'gradient_accumulation_steps': 1,
 'learning_rate': 1e-05,
 'load_best_model_at_end': True,
 'logging_dir': './logs',
 'logging_strategy': 'epoch',
 'lr_scheduler_type': 'cosine',
 'num_train_epochs': 20,
 'optim': 'adamw_torch',
 'overwrite_output_dir': True,
 'per_device_eval_batch_size': 2,
 'per_device_train_batch_size': 2,
 'report_to': 'wandb',
 'save_strategy': 'epoch',
 'save_total_limit': 5,
 'seed': 42,
 'warmup_ratio': 0.1,
 'weight_decay': 0.01}

In [11]:
# 모델 학습 과정에 대한 정보를 제공해주는 wandb 설정 내용을 확인합니다.
loaded_config['wandb']

{'entity': 'wandb_repo', 'name': 'run_name', 'project': 'project_name'}

In [12]:
# (선택) 이곳에 사용자가 사용할 wandb config 설정
loaded_config['wandb']['entity'] = "je_"
loaded_config['wandb']['name'] = "llama_type01"
loaded_config['wandb']['project'] = "nlp"

In [13]:
# 모델이 최종 결과를 출력하기 위한 매개변수 정보를 확인합니다.
loaded_config['inference']

{'batch_size': 4,
 'ckt_path': 'model ckt path',
 'early_stopping': True,
 'max_new_tokens': 100,
 'no_repeat_ngram_size': 2,
 'num_beams': 4,
 'remove_tokens': ['<usr>', '<s>', '</s>', '</s>'],
 'result_path': './prediction/'}

### 4) 데이터 불러와서 확인해보기
- 실험에서 쓰일 데이터를 load하여 데이터의 구조와 내용을 살펴보겠습니다.
- Train, dev, test 순서대로 12457, 499, 250개 씩 데이터가 구성되어 있습니다.

In [14]:
# config에 저장된 데이터 경로를 통해 train과 validation data를 불러옵니다.
data_path = loaded_config['general']['data_path']

# train data의 구조와 내용을 확인합니다.
train_df = pd.read_csv(os.path.join(data_path,'train.csv'))
train_df.tail()

,fname,dialogue,summary,topic
12452,train_12455,#Person1#: 실례합니다. 맨체스터 출신의 그린 씨이신가요?\n#Person2...,탄 링은 흰머리와 수염으로 쉽게 인식되는 그린 씨를 만나 호텔로 데려갈 예정입니다....,누군가를 태우다
12453,train_12456,#Person1#: 이윙 씨가 우리가 컨퍼런스 센터에 오후 4시에 도착해야 한다고 ...,#Person1#과 #Person2#는 이윙 씨가 늦지 않도록 요청했기 때문에 컨퍼...,컨퍼런스 센터
12454,train_12457,#Person1#: 오늘 어떻게 도와드릴까요?\n#Person2#: 차를 빌리고 싶...,#Person2#는 #Person1#의 도움으로 5일 동안 소형 차를 빌립니다.,차 렌트
12455,train_12458,#Person1#: 오늘 좀 행복해 보이지 않아. 무슨 일 있어?\n#Person2...,#Person2#의 엄마가 일자리를 잃었다. #Person2#는 엄마가 우울해하지 ...,실직
12456,train_12459,"#Person1#: 엄마, 다음 토요일에 이 삼촌네 가족을 방문하기 위해 비행기를 ...",#Person1#은 다음 토요일에 이 삼촌네를 방문할 때 가방을 어떻게 싸야 할지 ...,짐 싸기


In [15]:
# validation data의 구조와 내용을 확인합니다.
val_df = pd.read_csv(os.path.join(data_path,'dev.csv'))
val_df.tail()

,fname,dialogue,summary,topic
494,dev_495,#Person1#: 이제 새해가 되어서 새로운 시작을 하려고 결심했어. \r\n#P...,#Person1#은 새해에 금연을 하고 커밍아웃하기로 결정했습니다. #Person2...,새해
495,dev_496,"#Person1#: 너, 조랑 결혼했지? \r\n#Person2#: 조? 무슨 말인...",#Person1#은 #Person2#가 조와 결혼했다고 생각했다. #Person2#...,사랑에 빠지다
496,dev_497,"#Person1#: 무엇을 도와드릴까요, 부인?\r\n#Person2#: 몇 주 동...",#Person2#의 차에서 이상한 소리가 납니다. #Person1#는 브레이크를 교...,소음
497,dev_498,"#Person1#: 안녕하세요, 아마존 고객 서비스입니다. 무엇을 도와드릴까요?\n...",#Person2#님이 아마존 고객 서비스에 전화하여 아마존에서 받은 책에 한 페이지...,빠진 페이지
498,dev_499,#Person1#: 여름이 다 되어간다는 게 믿기지 않아.\r\n#Person2#:...,#Person2#는 #Person1#에게 여름 휴가 동안 파티를 도와주는 회사에서 ...,여름 휴가


## 1. 데이터 가공 및 데이터셋 클래스 구축
- csv file 을 불러와서 encoder 와 decoder의 입력형태로 가공해줍니다.
- 가공된 데이터를 torch dataset class 로 구축하여 모델에 입력가능한 형태로 만듭니다.

In [16]:
# 데이터 전처리를 위한 클래스로, 데이터셋을 데이터프레임으로 변환하고 인코더와 디코더의 입력을 생성합니다.
class Preprocess:
    def __init__(self,
            # bos_token: str,
            eos_token: str,
        ) -> None:

        # self.bos_token = bos_token
        self.eos_token = eos_token

    @staticmethod
    # 실험에 필요한 컬럼을 가져옵니다.
    def make_set_as_df(file_path, is_train = True):
        if is_train:
            df = pd.read_csv(file_path)
            train_df = df[['fname','dialogue','summary']]
            return train_df
        else:
            df = pd.read_csv(file_path)
            test_df = df[['fname','dialogue']]
            return test_df

    # BART 모델의 입력, 출력 형태를 맞추기 위해 전처리를 진행합니다.
    # def make_input(self, dataset,is_test = False):
    #     if is_test:
    #         encoder_input = dataset['dialogue'].apply(lambda x : 'summarize: ' + str(x))
    #         decoder_input = [' '] * len(dataset['dialogue'])
    #         return encoder_input.tolist(), list(decoder_input)
    #     else:
    #         encoder_input = dataset['dialogue'].apply(lambda x : 'summarize: ' + str(x))
    #         decoder_input = dataset['summary'].apply(lambda x : 'summarize: ' + str(x)) # Ground truth를 디코더의 input으로 사용하여 학습합니다.
    #         decoder_output = dataset['summary'].apply(lambda x : str(x) + self.eos_token)
    #         return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()
    def make_input(self, dataset, is_test=False):
        if is_test:
            # LLaMA 모델에 대한 입력 프롬프트 생성
            encoder_input = dataset['dialogue'].apply(lambda x: f"다음 텍스트를 텍스트 내부의 키워드를 사용하여 요약하세요.: {x}")
            # 테스트 데이터에서는 디코더 출력이 필요하지 않으므로 빈 문자열을 반환
            decoder_input = [''] * len(dataset['dialogue'])
            return encoder_input.tolist(), decoder_input
        else:
            # 학습 데이터에 대한 입력 프롬프트와 목표 출력 생성
            encoder_input = dataset['dialogue'].apply(lambda x: f"다음 텍스트를 텍스트 내부의 키워드를 사용하여 요약하세요.: {x}")
            decoder_input = dataset['summary'].apply(lambda x: f"Summary: {x}")  # 디코더 입력으로 사용할 목표 텍스트
            decoder_output = dataset['summary'].apply(lambda x: f"{x} {self.eos_token}")  # 목표 출력에 eos_token을 추가
            return encoder_input.tolist(), decoder_input.tolist(), decoder_output.tolist()


In [17]:
# Train에 사용되는 Dataset 클래스를 정의합니다.
# class DatasetForTrain(Dataset):
#     def __init__(self, encoder_input, decoder_input, labels, len):
#         self.encoder_input = encoder_input
#         self.decoder_input = decoder_input
#         self.labels = labels
#         self.len = len

#     def __getitem__(self, idx):
#         item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
#         item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
#         item2['decoder_input_ids'] = item2['input_ids']
#         item2['decoder_attention_mask'] = item2['attention_mask']
#         item2.pop('input_ids')
#         item2.pop('attention_mask')
#         item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
#         item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
#         return item

#     def __len__(self):
#         return self.len

# # Validation에 사용되는 Dataset 클래스를 정의합니다.
# class DatasetForVal(Dataset):
#     def __init__(self, encoder_input, decoder_input, labels, len):
#         self.encoder_input = encoder_input
#         self.decoder_input = decoder_input
#         self.labels = labels
#         self.len = len

#     def __getitem__(self, idx):
#         item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()} # item[input_ids], item[attention_mask]
#         item2 = {key: val[idx].clone().detach() for key, val in self.decoder_input.items()} # item2[input_ids], item2[attention_mask]
#         item2['decoder_input_ids'] = item2['input_ids']
#         item2['decoder_attention_mask'] = item2['attention_mask']
#         item2.pop('input_ids')
#         item2.pop('attention_mask')
#         item.update(item2) #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask]
#         item['labels'] = self.labels['input_ids'][idx] #item[input_ids], item[attention_mask] item[decoder_input_ids], item[decoder_attention_mask], item[labels]
#         return item

#     def __len__(self):
#         return self.len

# # Test에 사용되는 Dataset 클래스를 정의합니다.
# class DatasetForInference(Dataset):
#     def __init__(self, encoder_input, test_id, len):
#         self.encoder_input = encoder_input
#         self.test_id = test_id
#         self.len = len

#     def __getitem__(self, idx):
#         item = {key: val[idx].clone().detach() for key, val in self.encoder_input.items()}
#         item['ID'] = self.test_id[idx]
#         return item

#     def __len__(self):
#         return self.len


In [18]:
from torch.utils.data import Dataset

class DatasetForTrain(Dataset):
    def __init__(self, tokenizer, encoder_input, decoder_input, labels):
        self.tokenizer = tokenizer
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels

    def __getitem__(self, idx):
        encoder_text = self.encoder_input[idx]
        decoder_text = self.decoder_input[idx]
        labels_text = self.labels[idx]
        
        # 토큰화
        encoder_encodings = self.tokenizer(encoder_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        decoder_encodings = self.tokenizer(decoder_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        labels_encodings = self.tokenizer(labels_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        
        item = {key: torch.squeeze(val, 0) for key, val in encoder_encodings.items()}
        item.update({key: torch.squeeze(val, 0) for key, val in decoder_encodings.items()})
        item['labels'] = torch.squeeze(labels_encodings['input_ids'], 0)
        
        return item

    def __len__(self):
        return len(self.encoder_input)

class DatasetForVal(Dataset):
    def __init__(self, tokenizer, encoder_input, decoder_input, labels):
        self.tokenizer = tokenizer
        self.encoder_input = encoder_input
        self.decoder_input = decoder_input
        self.labels = labels

    def __getitem__(self, idx):
        encoder_text = self.encoder_input[idx]
        decoder_text = self.decoder_input[idx]
        labels_text = self.labels[idx]
        
        # 토큰화
        encoder_encodings = self.tokenizer(encoder_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        decoder_encodings = self.tokenizer(decoder_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        labels_encodings = self.tokenizer(labels_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        
        item = {key: torch.squeeze(val, 0) for key, val in encoder_encodings.items()}
        item.update({key: torch.squeeze(val, 0) for key, val in decoder_encodings.items()})
        item['labels'] = torch.squeeze(labels_encodings['input_ids'], 0)
        
        return item

    def __len__(self):
        return len(self.encoder_input)

class DatasetForInference(Dataset):
    def __init__(self, tokenizer, encoder_input, test_id):
        self.tokenizer = tokenizer
        self.encoder_input = encoder_input
        self.test_id = test_id

    def __getitem__(self, idx):
        encoder_text = self.encoder_input[idx]
        encodings = self.tokenizer(encoder_text, truncation=True, padding='max_length', max_length=512, return_tensors='pt')
        
        item = {key: torch.squeeze(val, 0) for key, val in encodings.items()}
        item['ID'] = self.test_id[idx]
        
        return item

    def __len__(self):
        return len(self.encoder_input)

In [19]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_train_dataset(config, preprocessor, data_path, tokenizer):
    train_file_path = os.path.join(data_path,'train.csv')
    val_file_path = os.path.join(data_path,'dev.csv')

    # train, validation에 대해 각각 데이터프레임을 구축합니다.
    train_data = preprocessor.make_set_as_df(train_file_path)
    val_data = preprocessor.make_set_as_df(val_file_path)

    print('-'*150)
    print(f'train_data:\n {train_data["dialogue"][0]}')
    print(f'train_label:\n {train_data["summary"][0]}')

    print('-'*150)
    print(f'val_data:\n {val_data["dialogue"][0]}')
    print(f'val_label:\n {val_data["summary"][0]}')

    encoder_input_train , decoder_input_train, decoder_output_train = preprocessor.make_input(train_data)
    encoder_input_val , decoder_input_val, decoder_output_val = preprocessor.make_input(val_data)
    print('-'*10, 'Load data complete', '-'*10,)

    tokenized_encoder_inputs = tokenizer(encoder_input_train, return_tensors="pt", padding=True,
                            add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_inputs = tokenizer(decoder_input_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    tokenized_decoder_ouputs = tokenizer(decoder_output_train, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    train_inputs_dataset = DatasetForTrain(tokenized_encoder_inputs, tokenized_decoder_inputs, tokenized_decoder_ouputs,len(encoder_input_train))

    val_tokenized_encoder_inputs = tokenizer(encoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_inputs = tokenizer(decoder_input_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)
    val_tokenized_decoder_ouputs = tokenizer(decoder_output_val, return_tensors="pt", padding=True,
                        add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False)

    val_inputs_dataset = DatasetForVal(val_tokenized_encoder_inputs, val_tokenized_decoder_inputs, val_tokenized_decoder_ouputs,len(encoder_input_val))

    print('-'*10, 'Make dataset complete', '-'*10,)
    return train_inputs_dataset, val_inputs_dataset

## 2. Trainer 및 Trainingargs 구축하기
- Huggingface 의 Trainer 와 Training arguments를 활용하여 모델 학습을 일괄적으로 처리해주는 클래스를 정의합니다.

In [20]:
# 모델 성능에 대한 평가 지표를 정의합니다. 본 대회에서는 ROUGE 점수를 통해 모델의 성능을 평가합니다.
def compute_metrics(config,tokenizer,pred):
    rouge = Rouge()
    predictions = pred.predictions
    labels = pred.label_ids

    predictions[predictions == -100] = tokenizer.pad_token_id
    labels[labels == -100] = tokenizer.pad_token_id

    decoded_preds = tokenizer.batch_decode(predictions, clean_up_tokenization_spaces=True)
    labels = tokenizer.batch_decode(labels, clean_up_tokenization_spaces=True)

    # 정확한 평가를 위해 미리 정의된 불필요한 생성토큰들을 제거합니다.
    replaced_predictions = decoded_preds.copy()
    replaced_labels = labels.copy()
    remove_tokens = config['inference']['remove_tokens']
    for token in remove_tokens:
        replaced_predictions = [sentence.replace(token," ") for sentence in replaced_predictions]
        replaced_labels = [sentence.replace(token," ") for sentence in replaced_labels]

    print('-'*150)
    print(f"PRED: {replaced_predictions[0]}")
    print(f"GOLD: {replaced_labels[0]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[1]}")
    print(f"GOLD: {replaced_labels[1]}")
    print('-'*150)
    print(f"PRED: {replaced_predictions[2]}")
    print(f"GOLD: {replaced_labels[2]}")

    # 최종적인 ROUGE 점수를 계산합니다.
    results = rouge.get_scores(replaced_predictions, replaced_labels,avg=True)

    # ROUGE 점수 중 F-1 score를 통해 평가합니다.
    result = {key: value["f"] for key, value in results.items()}
    return result

In [21]:
# 학습을 위한 trainer 클래스와 매개변수를 정의합니다.
def load_trainer_for_train(config,generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset):
    print('-'*10, 'Make training arguments', '-'*10,)
    # set training args
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=3,
        per_device_train_batch_size=1,  # 배치 크기를 줄임
        gradient_accumulation_steps=8,  # 기울기 누적 단계 증가
        fp16=True,  # Mixed Precision Training 사용
        per_device_eval_batch_size=1,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        save_steps=1000,  # 모델 저장 빈도 줄이기
        save_total_limit=2,
        evaluation_strategy="steps",
        eval_steps=500,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
    )

    # (선택) 모델의 학습 과정을 추적하는 wandb를 사용하기 위해 초기화 해줍니다.
    wandb.init(
        entity=config['wandb']['entity'],
        project=config['wandb']['project'],
        name=config['wandb']['name'],
    )

    # (선택) 모델 checkpoint를 wandb에 저장하도록 환경 변수를 설정합니다.
    # os.environ["WANDB_LOG_MODEL"]="true"
    # os.environ["WANDB_WATCH"]="false"

    # Validation loss가 더 이상 개선되지 않을 때 학습을 중단시키는 EarlyStopping 기능을 사용합니다.
    MyCallback = EarlyStoppingCallback(
        early_stopping_patience=config['training']['early_stopping_patience'],
        early_stopping_threshold=config['training']['early_stopping_threshold']
    )
    print('-'*10, 'Make training arguments complete', '-'*10,)
    print('-'*10, 'Make trainer', '-'*10,)

    # Trainer 클래스를 정의합니다.
    trainer = Seq2SeqTrainer(
        model=generate_model, # 사용자가 사전 학습하기 위해 사용할 모델을 입력합니다.
        args=training_args,
        train_dataset=train_inputs_dataset,
        eval_dataset=val_inputs_dataset,
        compute_metrics = lambda pred: compute_metrics(config,tokenizer, pred),
        callbacks = [MyCallback]
    )
    print('-'*10, 'Make trainer complete', '-'*10,)

    return trainer

In [22]:
# 학습을 위한 tokenizer와 사전 학습된 모델을 불러옵니다.
def load_tokenizer_and_model_for_train(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)
    print('-'*10, f'Model Name : {config["general"]["model_name"]}', '-'*10,)
    model_name = config['general']['model_name']
    bart_config = LlamaConfig().from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    generate_model = AutoModelForCausalLM.from_pretrained(config['general']['model_name'],config=bart_config)

    special_tokens_dict={'additional_special_tokens':config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model.resize_token_embeddings(len(tokenizer)) # 사전에 special token을 추가했으므로 재구성 해줍니다.
    generate_model.to(device)
    print(generate_model.config)

    print('-'*10, 'Load tokenizer & model complete', '-'*10,)
    return generate_model , tokenizer

## 3. 모델 학습하기

- 앞에서 구축한 클래스 및 함수를 활용하여 학습 진행합니다.

In [23]:
def main(config):
    # 사용할 device를 정의합니다.
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    # 사용할 모델과 tokenizer를 불러옵니다.
    generate_model , tokenizer = load_tokenizer_and_model_for_train(config,device)
    print('-'*10,"tokenizer special tokens : ",tokenizer.special_tokens_map,'-'*10)

    # 학습에 사용할 데이터셋을 불러옵니다.
    preprocessor = Preprocess(config['tokenizer']['eos_token']) # decoder_start_token: str, eos_token: str
    data_path = config['general']['data_path']
    train_inputs_dataset, val_inputs_dataset = prepare_train_dataset(config,preprocessor, data_path, tokenizer)

    # Trainer 클래스를 불러옵니다.
    trainer = load_trainer_for_train(config, generate_model,tokenizer,train_inputs_dataset,val_inputs_dataset)
    trainer.train()   # 모델 학습을 시작합니다.

    # (선택) 모델 학습이 완료된 후 wandb를 종료합니다.
    wandb.finish()

In [24]:
if __name__ == "__main__":
    main(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : beomi/llama-2-ko-7b ----------


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 48.81 MiB is free. Process 256824 has 23.63 GiB memory in use. Of the allocated memory 23.38 GiB is allocated by PyTorch, and 2.95 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## 4. 모델 추론하기

In [46]:
# 이곳에 내가 사용할 wandb config 설정
loaded_config['inference']['ckt_path'] = "../output/t5_type08/checkpoint-1000"

- test data를 사용하여 모델의 성능을 확인합니다.

In [36]:
# tokenization 과정까지 진행된 최종적으로 모델에 입력될 데이터를 출력합니다.
def prepare_test_dataset(config,preprocessor, tokenizer):

    test_file_path = os.path.join(config['general']['data_path'],'test.csv')

    test_data = preprocessor.make_set_as_df(test_file_path,is_train=False)
    test_id = test_data['fname']

    print('-'*150)
    print(f'test_data:\n{test_data["dialogue"][0]}')
    print('-'*150)

    encoder_input_test , decoder_input_test = preprocessor.make_input(test_data,is_test=True)
    print('-'*10, 'Load data complete', '-'*10,)

    test_tokenized_encoder_inputs = tokenizer(encoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['encoder_max_len'], return_token_type_ids=False,)
    test_tokenized_decoder_inputs = tokenizer(decoder_input_test, return_tensors="pt", padding=True,
                    add_special_tokens=True, truncation=True, max_length=config['tokenizer']['decoder_max_len'], return_token_type_ids=False,)

    test_encoder_inputs_dataset = DatasetForInference(test_tokenized_encoder_inputs, test_id, len(encoder_input_test))
    print('-'*10, 'Make dataset complete', '-'*10,)

    return test_data, test_encoder_inputs_dataset

In [31]:
# 추론을 위한 tokenizer와 학습시킨 모델을 불러옵니다.
def load_tokenizer_and_model_for_test(config,device):
    print('-'*10, 'Load tokenizer & model', '-'*10,)

    model_name = config['general']['model_name']
    ckt_path = config['inference']['ckt_path']
    print('-'*10, f'Model Name : {model_name}', '-'*10,)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    special_tokens_dict = {'additional_special_tokens': config['tokenizer']['special_tokens']}
    tokenizer.add_special_tokens(special_tokens_dict)

    generate_model = AutoModelForCausalLM.from_pretrained(ckt_path)
    generate_model.resize_token_embeddings(len(tokenizer))
    generate_model.to(device)
    print('-'*10, 'Load tokenizer & model complete', '-'*10,)

    return generate_model , tokenizer

In [45]:
# 학습된 모델이 생성한 요약문의 출력 결과를 보여줍니다.
def inference(config):
    device = torch.device('cuda:0' if torch.cuda.is_available()  else 'cpu')
    print('-'*10, f'device : {device}', '-'*10,)
    print(torch.__version__)

    generate_model , tokenizer = load_tokenizer_and_model_for_test(config,device)

    data_path = config['general']['data_path']
    preprocessor = Preprocess(config['tokenizer']['eos_token'])

    test_data, test_encoder_inputs_dataset = prepare_test_dataset(config,preprocessor, tokenizer)
    dataloader = DataLoader(test_encoder_inputs_dataset, batch_size=config['inference']['batch_size'])

    summary = []
    text_ids = []
    with torch.no_grad():
        for item in tqdm(dataloader):
            text_ids.extend(item['ID'])
            generated_ids = generate_model.generate(input_ids=item['input_ids'].to('cuda:0'),
                            no_repeat_ngram_size=config['inference']['no_repeat_ngram_size'],
                            early_stopping=config['inference']['early_stopping'],
                            max_length=config['inference']['generate_max_length'],
                            num_beams=config['inference']['num_beams'],
                        )
            for ids in generated_ids:
                result = tokenizer.decode(ids)
                summary.append(result)

    # 정확한 평가를 위하여 노이즈에 해당되는 스페셜 토큰을 제거합니다.
    remove_tokens = config['inference']['remove_tokens']
    preprocessed_summary = summary.copy()
    for token in remove_tokens:
        preprocessed_summary = [sentence.replace(token," ") for sentence in preprocessed_summary]

    output = pd.DataFrame(
        {
            "fname": test_data['fname'],
            "summary" : preprocessed_summary,
        }
    )
    result_path = config['inference']['result_path']
    if not os.path.exists(result_path):
        os.makedirs(result_path)
    output.to_csv(os.path.join(result_path, "output_t5_type08'1000.csv"), index=False)

    return output

In [47]:
# 학습된 모델의 test를 진행합니다.
if __name__ == "__main__":
    output = inference(loaded_config)

---------- device : cuda:0 ----------
2.1.0
---------- Load tokenizer & model ----------
---------- Model Name : eenzeenee/t5-small-korean-summarization ----------
---------- Load tokenizer & model complete ----------
------------------------------------------------------------------------------------------------------------------------------------------------------
test_data:
#Person1#: 더슨 씨, 받아쓰기 좀 해주세요. 
#Person2#: 네, 실장님...
#Person1#: 이것은 오늘 오후까지 모든 직원에게 내부 메모로 전달되어야 합니다. 준비되셨나요?
#Person2#: 네, 실장님. 시작하셔도 됩니다.
#Person1#: 모든 직원들에게 주의하라... 즉시 효력을 발휘하여, 모든 사무실 통신은 이메일 통신과 공식 메모로 제한됩니다. 근무 시간 동안 직원들이 즉시 메시지 프로그램을 사용하는 것은 엄격히 금지됩니다.
#Person2#: 실장님, 이것은 내부 통신에만 적용되는 건가요? 아니면 외부 통신에도 제한이 되는 건가요?
#Person1#: 이것은 모든 통신에 적용되어야 합니다, 이 사무실 내의 직원들 사이뿐만 아니라 외부 통신에도 마찬가지입니다.
#Person2#: 하지만 실장님, 많은 직원들이 고객과 소통하기 위해 즉시 메시지를 사용하고 있습니다.
#Person1#: 그들은 그들의 의사소통 방법을 바꾸어야만 합니다. 이 사무실에서 누구도 즉시 메시지를 사용하지 않기를 원합니다. 너무 많은 시간을 낭비하게 됩니다! 이제, 메모를 계속해주세요. 우리가 어디까지 했나요?
#Person2#: 이것은 내부와 외부 통신에 적용됩니다.
#Person1#: 

100%|██████████| 16/16 [00:47<00:00,  2.99s/it]


In [48]:
output  # 각 대화문에 대한 요약문이 출력됨을 확인할 수 있습니다.

,fname,summary
0,test_0,실장은 모든 직원에게 즉시 메시지 프로그램 사용을 하도록 하고 업무 시간 동안 지...
1,test_1,버스는 교통 체증에 걸려 지하철을 타고 가는 것이 더 좋을것 같아 버스를 는것은...
2,test_2,케이트가 2개월 동안 별거 중이다가 이혼을 신청했고 아이들은 어떻게 되는 것 같아...
3,test_3,생일 축하하고 파티를 즐겨. 생일을 위해 이 씨에게 와 춤에 대해 는 것 을은 ...
4,test_4,올림픽 공원의 중심인 올림픽공원은 6월에 완공될 예정이다. 이공원 에서 관람석 몇...
...,...,...
494,test_495,잭은 학교 끝나고 비디오 게임을 하고 싶다고 했다. 잭에게 이게임 를 는 것 에...
495,test_496,아내와 처음으로 레코드 플레이어를 구입했을 때 컨트리 음악에 관심을 가지게 된 것...
496,test_497,앨리스는 기계를 어떻게 사용하는지 알려 줄 수 있는지 을은 세탁기 에 대해 의 ...
497,test_498,다우 부인에게 아파트를 보여줄 수 있는지 알려 달라고 은 씨 에게. 에 는 이 ...
